<a href="https://colab.research.google.com/github/tlysenko/bionformatics-coursera-sandiego/blob/main/Genome%20Sequencing%20(Bioinformatics%20II)/Bioinformatics_2_week_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing libraries

In [47]:
import numpy as np
import pandas as pd

from google.colab import files
import urllib.request

from collections import defaultdict
from collections import Counter


import matplotlib.pyplot as plt

## 1.1.1 Cyclopeptide Scoring Problem: Compute the score of a cyclic peptide against a spectrum.

Input: An amino acid string Peptide and a collection of integers Spectrum.

Output: The score of Peptide against Spectrum, Score(Peptide, Spectrum).

In [14]:
def Cyclospectrum(peptide):
  def LinearSpectrum(peptide, alphabet, aminoAcidMass):
    
    lPeptide = len(peptide)
    prefixMass = [0] * (lPeptide) 
    for i in range(0, lPeptide):
      for symbol in alphabet:
        if symbol == peptide[i]:
          prefixMass[i] = prefixMass[i-1] + aminoAcidMass[symbol]
    prefixMass.insert(0,0)
    linearSpectrum = [0]

    c = 0
    for i in range(0, len(prefixMass)-1):
      for j in range(i+1, len(prefixMass)):
        c +=1
        linearSpectrum.append(prefixMass[j]-prefixMass[i])

    linearSpectrum.sort()
    return linearSpectrum

  # Download the data
  with open('/content/drive/MyDrive/Colab Notebooks/Bionformatics-coursera-sandiego/Course 2/integer_mass_table.txt') as f:
    tb = f.readlines()
  massTable = dict()
  for line in tb:
    massTable[line.split('\n')[0].split(' ')[0]] = int(line.split('\n')[0].split(' ')[1])
    
  # *** Main ***  

  part1 = LinearSpectrum(peptide=peptide[:-1], alphabet=massTable.keys(), aminoAcidMass=massTable)
  pref1 = LinearSpectrum(peptide=peptide, alphabet=massTable.keys(), aminoAcidMass=massTable)
  tMass=pref1[-1] # mass of the full element
  part2 = [tMass - x for x in part1]

  final = part1 + part2
  final.sort()
  return final

Cyclospectrum(peptide='LEQN')

[0, 113, 114, 128, 129, 227, 242, 242, 257, 355, 356, 370, 371, 484]

In [41]:
def Score(peptide, spectrum):
  db = GetAminoAcidMassTable()
  CS = Cyclospectrum(peptide)
  CP = spectrum.copy()
  score = 0
  for s in CS:
    if (s in CP):
      score += 1 
      CP.remove(s)
  return score

peptide = 'NQEL'
spectrum = '0 99 113 114 128 227 257 299 355 356 370 371 484'
spectrum = [int(s) for s in spectrum.split(' ')]
Score(peptide, spectrum)

11

### Extra Dataset

In [ ]:
url = 'http://bioinformaticsalgorithms.com/data/extradatasets/antibiotics/cyclopeptide_scoring.txt'
with urllib.request.urlopen(url) as f:
    data = f.read().decode('utf-8')

In [35]:
peptide = data.split('\r\n')[1]
spectrum = [int(s) for s in data.split('\r\n')[2].split(' ')]
print(peptide)
print(spectrum[:10])

print('score: ', Score(peptide, spectrum))

VYYEVDWTMGRQIDPDEYPIAQCTRHRATILTLPDWQM
[0, 71, 71, 87, 87, 97, 97, 99, 101, 101]
score:  521


### Stepik submission

In [36]:
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving dataset_102_3-3.txt to dataset_102_3-3.txt
User uploaded file "dataset_102_3-3.txt" with length 7925 bytes


In [37]:
with open(fn) as f:
  peptide = f.readline().split('\n')[0]
  spectr = f.readline().split('\n')[0]
spectr = [int(s) for s in spectr.split(' ')]

print(spectr)
print(peptide)
print('score: ', Score(peptide, spectr))

[0, 71, 71, 87, 87, 97, 97, 99, 99, 99, 101, 101, 101, 103, 103, 113, 113, 114, 114, 115, 115, 115, 115, 128, 128, 128, 129, 131, 131, 137, 137, 137, 147, 147, 147, 147, 147, 156, 156, 163, 168, 172, 186, 186, 198, 198, 201, 202, 204, 208, 212, 213, 218, 218, 218, 224, 227, 228, 229, 229, 232, 246, 246, 248, 250, 252, 260, 260, 262, 262, 265, 268, 269, 269, 270, 271, 275, 284, 284, 291, 295, 299, 299, 300, 301, 301, 303, 316, 317, 319, 329, 332, 333, 339, 355, 355, 355, 355, 357, 358, 359, 360, 361, 363, 364, 365, 366, 370, 376, 378, 383, 385, 390, 390, 392, 396, 398, 398, 404, 412, 415, 416, 416, 418, 418, 426, 430, 431, 432, 438, 446, 447, 454, 454, 458, 460, 461, 464, 467, 470, 472, 483, 491, 492, 493, 495, 495, 497, 497, 502, 505, 511, 513, 517, 518, 519, 525, 527, 529, 532, 532, 533, 534, 545, 545, 547, 551, 553, 555, 559, 561, 561, 562, 567, 568, 571, 582, 584, 592, 594, 598, 601, 601, 607, 614, 616, 618, 619, 619, 620, 620, 624, 626, 630, 640, 647, 648, 649, 655, 655, 656, 656, 

## 1.1.2 LeaderboardCyclopeptideSequencing

Code Challenge: Implement LeaderboardCyclopeptideSequencing.

Input: An integer N and a collection of integers Spectrum.
Output: LeaderPeptide after running LeaderboardCyclopeptideSequencing(Spectrum, N).


ToDO:

- Trim function and issue with connected trims. Connected means has similar score
- Change scoring from cycle to linear

In [ ]:
# LeaderboardCyclopeptideSequencing(Spectrum, N)
#     Leaderboard ← set containing only the empty peptide
#     LeaderPeptide ← empty peptide
#     while Leaderboard is non-empty
#         Leaderboard ← Expand(Leaderboard)
#         for each Peptide in Leaderboard
#             if Mass(Peptide) = ParentMass(Spectrum)
#                 if Score(Peptide, Spectrum) > Score(LeaderPeptide, Spectrum)
#                     LeaderPeptide ← Peptide
#             else if Mass(Peptide) > ParentMass(Spectrum)
#                 remove Peptide from Leaderboard
#         Leaderboard ← Trim(Leaderboard, Spectrum, N)
#     output LeaderPeptide

In [120]:
def LeaderBoard(n, spectrum):

  def LinearSpectrum(peptide):
    #print('peptide', peptide)
    
    lPeptide = len(peptide)
    prefixMass = [0] * (lPeptide) 

    for i in range(0, lPeptide):
      for symbol in tbAAtoMass.keys():
        if symbol == peptide[i]:
          prefixMass[i] = prefixMass[i-1] + tbAAtoMass[symbol]
    prefixMass.insert(0,0)
    
    linearSpectrum = [0]
    c = 0
    for i in range(0, len(prefixMass)-1):
      for j in range(i+1, len(prefixMass)):
        c +=1
        linearSpectrum.append(prefixMass[j]-prefixMass[i])

    linearSpectrum.sort()
    #print(linearSpectrum)
    return linearSpectrum


  def GetAminoAcidMassTable():
    ''' Returns dictionary in format aminoacid: integer mass  
    '''
    with open('/content/drive/MyDrive/Colab Notebooks/Bionformatics-coursera-sandiego/Course 2/integer_mass_table.txt') as f:
      tb = f.readlines()
    massTable = dict()
    for line in tb:
      massTable[line.split('\n')[0].split(' ')[0]] = int(line.split('\n')[0].split(' ')[1])
    return massTable


  def GetMassAminoAcidTable():
    ''' Returns dictionary in format integer mass : aminoacid
    '''
    with open('/content/drive/MyDrive/Colab Notebooks/Bionformatics-coursera-sandiego/Course 2/integer_mass_table.txt') as f:
      tb = f.readlines()
    massTable = dict()
    for line in tb:
      massTable[int(line.split('\n')[0].split(' ')[1])] = line.split('\n')[0].split(' ')[0]
    return massTable


  def Expand(candidatePeptides):
      ''' Returns newCollection containing all possible extensions of peptides in 
          candidatePeptides by a single amino acid mass
      ''' 
      newCollection = set()
      if candidatePeptides != {''}:
        
        [newCollection.add(candidate + '-' + str(tbAAtoMass[peptide])) 
                          for candidate in candidatePeptides \
                            for peptide in tbAAtoMass  ]
      else:
        [newCollection.add(candidate + str(tbAAtoMass[peptide])) \
                          for candidate in candidatePeptides \
                          for peptide in tbAAtoMass ]
                        
      return newCollection


  def GetParentMass(spectrum):
    
    return spectrum[-1]
  

  def GetMass(peptide):
    ''' Input: subpeptide string
        Output: mass of subpeptidestring
    '''
    mass = 0 
    
    for element in peptide.split('-'):
      if element != '':
        mass = mass + int(element)

    return mass 


  def PeptToAA(peptide):
    ''' input: aminoacid sequence in format 137-57-128
        output: aminoacid
        TODO: combinations with similar AAs 
        113:'I/L',
        128: 'K/Q'
    '''
    aminoAcid = ''
    for aa in peptide.split('-'):
      aminoAcid += tbMasstoAA[int(aa)]
    return aminoAcid 


  def PeptideConsistentWithSpectrum(peptide):
    ''' case 1: peptide is string '128'
        case 2: peptide is string '128-113-99'
    '''
    peptideAA = PeptToAA(peptide) 
    LS = LinearSpectrum(peptide=peptideAA, alphabet=massTable.keys(), aminoAcidMass=massTable)

    spectrumCopy = spectrum.copy()
    cnt = 0

    for element in LS:
      if int(element) in spectrumCopy:
        cnt += 1 
        spectrumCopy.remove(int(element))
      else:
        return False
    if cnt == len(LS):
      return True
      

  def LinearScore(peptide, spectrum):
    LS = LinearSpectrum(peptide)
    CP = spectrum.copy()
    score = 0
    #print('LS', peptide, LS)

    if peptide != '':
      for s in LS:
        if (s in CP):
          score += 1 
          CP.remove(s)

    return score


  def Trim(leaderBoard, spectrum, n):
    ''' leaderBoard: set of peptides {'99-99-99', ''}
        spectrum: list of int [0,99,128]
        n = 10 
    '''

    if len(leaderBoard) > n:
      scores = []
      trimedLeaderBoard = set()

      [scores.append(LinearScore(PeptToAA(peptide), spectrum)) for peptide in leaderBoard]
      #print('scores', scores)
      
      sortedLB = [l for _,l in sorted(zip(scores,leaderBoard))]
      #print('sortedLB', sortedLB)
      
      scores = sorted(scores)
      #print('sorted scores', scores)

      part1 = sortedLB[-n:]
      #print('part1', part1)
      
      part2 = []
      i = 1
      while len(scores)>(n+i) and (scores[-n]==scores[-n-i]):
        part2.append(sortedLB[-n-i])
        i += 1
      trimedLeaderBoard = set(part1 + part2)
    else: 
      trimedLeaderBoard = leaderBoard
    #print('trimedLeaderBoard ', len(leaderBoard), '->',len(trimedLeaderBoard),  trimedLeaderBoard )
    return trimedLeaderBoard 


  # *** Main *** 

  tbAAtoMass = GetAminoAcidMassTable()
  tbMasstoAA = GetMassAminoAcidTable()
  parentMass = GetParentMass(spectrum)

  leaderBoard = {''} # set of strings
  leaderPeptide = ''

  while len(leaderBoard) > 0:
    leaderBoardExp = Expand(leaderBoard)
    leaderBoardCopy = leaderBoardExp.copy()

    for peptide in leaderBoardExp:  # peptide 137-57-128
      
      peptideMass = GetMass(peptide)
      if peptideMass == parentMass:
        peptideAA = PeptToAA(peptide)
        LS1 = LinearScore(peptideAA, spectrum)
        LS2 = LinearScore(leaderPeptide, spectrum)
        if  LS1 > LS2 :
          leaderPeptide = peptide 
      elif peptideMass > parentMass:
        leaderBoardCopy.remove(peptide)
    leaderBoard = Trim(leaderBoardCopy, spectrum, n)

  return leaderPeptide

n = 10 
spectrum = [int(s) for s in '0 71 113 129 147 200 218 260 313 331 347 389 460'.split(' ')]
LB = LeaderBoard(n, spectrum)
print(LB)

129-71-113-147


### Extra dataset

In [48]:
url = 'http://bioinformaticsalgorithms.com/data/extradatasets/antibiotics/leaderboard_cyclopeptide_sequencing.txt'
with urllib.request.urlopen(url) as f:
    data = f.read().decode('utf-8')

In [49]:
data.split('\r\n')[4]

'97-129-97-147-99-71-186-71-113-163-115-71-113-128-103-87-128-101-137-163-114'

In [50]:
n1 = int(data.split('\r\n')[1])
spectrum1 = [int(s) for s in data.split('\r\n')[2].split(' ')]
print(n1, spectrum1[:10])

325 [0, 71, 71, 71, 87, 97, 97, 99, 101, 103]


In [52]:
l = LeaderBoard(n1, spectrum1)
l

'99-147-97-129-97-114-163-137-101-128-87-103-128-113-71-115-163-113-71-115-71-71'

### Stepik submission

In [54]:
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving dataset_102_8-2.txt to dataset_102_8-2.txt
User uploaded file "dataset_102_8-2.txt" with length 2362 bytes


In [55]:
with open(fn) as f:
  n = int(f.readline().split('\n')[0])
  spectr = f.readline().split('\n')[0]
spectr = [int(s) for s in spectr.split(' ')]

print(spectr)
print(n)

[0, 71, 87, 87, 97, 99, 113, 115, 115, 128, 128, 128, 128, 128, 131, 137, 137, 147, 147, 147, 147, 163, 186, 186, 202, 202, 214, 215, 215, 218, 234, 241, 243, 246, 252, 256, 257, 260, 265, 283, 284, 289, 291, 291, 294, 294, 314, 330, 330, 333, 346, 351, 361, 369, 378, 378, 380, 380, 381, 383, 385, 388, 388, 407, 417, 419, 420, 430, 441, 442, 458, 467, 475, 479, 480, 480, 493, 495, 498, 498, 506, 506, 509, 513, 516, 516, 535, 554, 555, 567, 567, 577, 580, 582, 593, 594, 595, 595, 603, 603, 621, 621, 626, 626, 627, 635, 637, 644, 663, 666, 681, 682, 702, 708, 708, 710, 714, 714, 724, 724, 731, 731, 732, 740, 741, 749, 757, 758, 763, 766, 773, 781, 795, 809, 810, 813, 828, 836, 837, 839, 844, 859, 860, 861, 861, 868, 873, 878, 886, 894, 894, 904, 910, 918, 920, 928, 956, 960, 965, 965, 967, 972, 972, 973, 974, 975, 981, 991, 996, 996, 1001, 1008, 1022, 1022, 1038, 1046, 1051, 1062, 1065, 1067, 1075, 1088, 1093, 1093, 1096, 1100, 1102, 1104, 1107, 1109, 1119, 1119, 1121, 1135, 1138, 1150, 

In [56]:
l = LeaderBoard(n, spectr)

In [57]:
l

'97-137-147-99-115-137-128-115-87-128-163-128-87-131-71-186-128-128-113-147-147-147-99-87'

## Noisy spectrums

In [181]:
def LeaderBoardTop(n, spectrum):

  def LinearSpectrum(peptide):
    #print('peptide', peptide)
    
    lPeptide = len(peptide)
    prefixMass = [0] * (lPeptide) 

    for i in range(0, lPeptide):
      for symbol in tbAAtoMass.keys():
        if symbol == peptide[i]:
          prefixMass[i] = prefixMass[i-1] + tbAAtoMass[symbol]
    prefixMass.insert(0,0)
    
    linearSpectrum = [0]
    c = 0
    for i in range(0, len(prefixMass)-1):
      for j in range(i+1, len(prefixMass)):
        c +=1
        linearSpectrum.append(prefixMass[j]-prefixMass[i])

    linearSpectrum.sort()
    #print(linearSpectrum)
    return linearSpectrum

  def Cyclospectrum(peptide):
 
    part1 = LinearSpectrum(peptide=peptide[:-1])
    pref1 = LinearSpectrum(peptide=peptide)
    tMass=pref1[-1] # mass of the full element
    part2 = [tMass - x for x in part1]

    final = part1 + part2
    final.sort()
    return final


  def GetAminoAcidMassTable():
    ''' Returns dictionary in format aminoacid: integer mass  
    '''
    with open('/content/drive/MyDrive/Colab Notebooks/Bionformatics-coursera-sandiego/Course 2/integer_mass_table.txt') as f:
      tb = f.readlines()
    massTable = dict()
    for line in tb:
      massTable[line.split('\n')[0].split(' ')[0]] = int(line.split('\n')[0].split(' ')[1])
    return massTable


  def GetMassAminoAcidTable():
    ''' Returns dictionary in format integer mass : aminoacid
    '''
    with open('/content/drive/MyDrive/Colab Notebooks/Bionformatics-coursera-sandiego/Course 2/integer_mass_table.txt') as f:
      tb = f.readlines()
    massTable = dict()
    for line in tb:
      massTable[int(line.split('\n')[0].split(' ')[1])] = line.split('\n')[0].split(' ')[0]
    return massTable


  def Expand(candidatePeptides):
      ''' Returns newCollection containing all possible extensions of peptides in 
          candidatePeptides by a single amino acid mass
      ''' 
      newCollection = set()
      if candidatePeptides != {''}:
        
        [newCollection.add(candidate + '-' + str(tbAAtoMass[peptide])) 
                          for candidate in candidatePeptides \
                            for peptide in tbAAtoMass ]
      else:
        [newCollection.add(candidate + str(tbAAtoMass[peptide])) \
                          for candidate in candidatePeptides \
                          for peptide in tbAAtoMass ]
                        
      return newCollection


  def GetParentMass(spectrum):
    
    return spectrum[-1]
  

  def GetMass(peptide):
    ''' Input: subpeptide string
        Output: mass of subpeptidestring
    '''
    mass = 0 
    
    for element in peptide.split('-'):
      if element != '':
        mass = mass + int(element)

    return mass 


  def PeptToAA(peptide):
    ''' input: aminoacid sequence in format 137-57-128
        output: aminoacid
    '''
    aminoAcid = ''
    for aa in peptide.split('-'):
      aminoAcid += tbMasstoAA[int(aa)]
    return aminoAcid 


  def PeptideConsistentWithSpectrum(peptide):
    ''' case 1: peptide is string '128'
        case 2: peptide is string '128-113-99'
    '''
    peptideAA = PeptToAA(peptide) 
    LS = LinearSpectrum(peptide=peptideAA, alphabet=massTable.keys(), aminoAcidMass=massTable)

    spectrumCopy = spectrum.copy()
    cnt = 0

    for element in LS:
      if int(element) in spectrumCopy:
        cnt += 1 
        spectrumCopy.remove(int(element))
      else:
        return False
    if cnt == len(LS):
      return True
      

  def LinearScore(peptide, spectrum):
    LS = LinearSpectrum(peptide)
    CP = spectrum.copy()
    score = 0
    #print('LS', peptide, LS)

    if peptide != '':
      for s in LS:
        if (s in CP):
          score += 1 
          CP.remove(s)

    return score

  def CycloScore(peptide, spectrum):
    LS = Cyclospectrum(peptide)
    CP = spectrum.copy()
    score = 0
    #print('LS', peptide, LS)

    if peptide != '':
      for s in LS:
        if (s in CP):
          score += 1 
          CP.remove(s)

    return score


  def Trim(leaderBoard, spectrum, n):
    ''' leaderBoard: set of peptides {'99-99-99', ''}
        spectrum: list of int [0,99,128]
        n = 10 
    '''
    scores = []
    if len(leaderBoard) > n:
      
      trimedLeaderBoard = set()

      [scores.append(LinearScore(PeptToAA(peptide), spectrum)) for peptide in leaderBoard]
      #print('scores', sorted(scores,reverse=True))
      
      sortedLB = [l for _,l in sorted(zip(scores,leaderBoard))]
      #print('sortedLB', sortedLB)
      
      scores = sorted(scores)
      #print('sorted scores', scores)

      part1 = sortedLB[-n:]
      #print('part1', part1)
      
      part2 = []
      i = 1
      while len(scores)>(n+i) and (scores[-n]==scores[-n-i]):
        part2.append(sortedLB[-n-i])
        i += 1
      trimedLeaderBoard = set(part1 + part2)
    else: 
      trimedLeaderBoard = leaderBoard
    #print('trimedLeaderBoard ', len(leaderBoard), '->',len(trimedLeaderBoard),  trimedLeaderBoard )
    #print('scores',scores)
    return trimedLeaderBoard 


  # *** Main *** 

  tbAAtoMass = GetAminoAcidMassTable()
  tbMasstoAA = GetMassAminoAcidTable()
  parentMass = GetParentMass(spectrum)

  leaderBoard = {''} # set of strings
  leaderPeptide = ''

  topScore = 0
  topPeptides = []
  topScores = [] 


  while len(leaderBoard) > 0:
    leaderBoardExp = Expand(leaderBoard)
    leaderBoardCopy = leaderBoardExp.copy()

    for peptide in leaderBoardExp:  # peptide 137-57-128  
      peptideMass = GetMass(peptide)
      if peptideMass == parentMass:
        CScore1 = CycloScore(PeptToAA(peptide), spectrum)
        CScore2 = CycloScore(leaderPeptide, spectrum)
      
        if CScore1 == topScore:
          topPeptides.append(peptide) ###
          topScores.append(topScore)
        if topScore < CScore1:
          topPeptides = [peptide]
          topScores = [CScore1]
          topScore = CScore1
      
        if  CScore1 > CScore2 :
          leaderPeptide = peptide 
      elif peptideMass > parentMass:
        leaderBoardCopy.remove(peptide)
    leaderBoard = Trim(leaderBoardCopy, spectrum, n)
    

  return topPeptides #leaderPeptide, PeptToAA(leaderPeptide), CycloScore(PeptToAA(leaderPeptide), spectrum)

n = 10 
spectrum = [int(s) for s in '0 71 113 129 147 200 218 260 313 331 347 389 460'.split(' ')]
LB = LeaderBoardTop(n, spectrum)
print(LB)

topScore 13
len(topScores) 8
[13, 13, 13, 13, 13, 13, 13, 13]
len(set(topPeptides)) 8
>>>> I have found 8 peptides of score 13
['113-147-71-129', '71-129-113-147', '129-113-147-71', '129-71-147-113', '147-71-129-113', '71-147-113-129', '113-129-71-147', '147-113-129-71']


### Spectrum 10

In [153]:
url = 'http://bioinformaticsalgorithms.com/data/realdatasets/Antibiotics/Tyrocidine_B1_Spectrum_10.txt'

with urllib.request.urlopen(url) as f:
    data_10 = f.read().decode('utf-8')

data_10 = [int(x) for x in data_10.split(' ')]

data_10[:10]

[0, 97, 99, 114, 128, 147, 147, 163, 186, 227]

In [175]:
LB = LeaderBoardTop(n=1000, spectrum=data_10)
print(LB)

topScore 86
len(topScores) 240
[86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86]
len(set(

In [ ]:
# Applying LeaderboardCyclopeptideSequencing to this spectrum (with N = 1000) results in 
# the correct cyclic peptide VKLFPWFNQY, which has a score of 86.

### Spectrum 25

In [158]:
url = 'http://bioinformaticsalgorithms.com/data/realdatasets/Antibiotics/Tyrocidine_B1_Spectrum_25.txt'
with urllib.request.urlopen(url) as f:
    data_25 = f.read().decode('utf-8')

data_25 = [int(x) for x in data_25.split(' ')]

data_25[:10]

[0, 97, 99, 113, 114, 115, 128, 128, 147, 147]

In [176]:
LBT = LeaderBoardTop(n=1000, spectrum = data_25)
LBT 

topScore 83
len(topScores) 38
[83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83]
len(set(topPeptides)) 38
>>>> I have found 38 peptides of score 83


('97-147-113-128-99-163-71-57-57-57-147-115-71', 'PFLQVYAGGGFDA', 83)

In [ ]:
#  You should find 38 linear peptides of maximum score 83 
#  (corresponding to 15 different cyclic peptides)

## 1.1.3 Run LeaderboardCyclopeptideSequencing on Spectrum25 with N = 1000. 

You should find 38 linear peptides of maximum score 83 (corresponding to 15 different cyclic peptides). What are they? (Return your peptides in integer format, with each peptide separated by a single space, e.g., 113-147-71-129 71-147-129-113.)

In [186]:
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving dataset_102_10.txt to dataset_102_10 (1).txt
User uploaded file "dataset_102_10.txt" with length 398 bytes


In [187]:
with open(fn) as f:
  data = f.readlines()

n = data[0].split('\n')[0]
spectrum = data[1].split('\n')[0]

In [188]:
C = LeaderBoardTop(n=1000, spectrum = data_25)

topScore 83
len(topScores) 38
[83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83]
len(set(topPeptides)) 38
>>>> I have found 38 peptides of score 83


In [189]:
s = ''
for el in C:
  s += str(el) + ' '
s 

'97-147-113-128-99-163-128-114-147-71-115 128-99-163-128-114-147-115-71-97-147-113 115-71-147-114-128-163-99-128-113-147-97 115-147-114-128-163-99-128-113-147-97-71 128-114-147-115-71-97-147-113-128-99-163 128-99-163-128-114-147-71-115-97-147-113 163-128-114-147-115-71-97-147-113-128-99 114-128-163-99-128-113-147-97-71-115-147 97-147-113-128-99-163-128-114-147-115-71 97-71-115-147-114-128-163-99-128-113-147 114-128-163-99-128-113-147-97-115-71-147 99-163-128-114-147-115-71-97-147-113-128 113-128-99-163-128-114-147-115-71-97-147 147-113-128-99-163-128-114-147-71-115-97 113-128-99-163-128-114-147-71-115-97-147 99-163-128-114-147-71-115-97-147-113-128 147-114-128-163-99-128-113-147-97-115-71 147-113-128-99-163-128-114-147-115-71-97 71-115-147-114-128-163-99-128-113-147-97 147-114-128-163-99-128-113-147-97-71-115 147-114-128-163-99-57-71-113-147-97-115-71 97-147-113-128-99-163-128-57-57-147-71-115 147-114-128-163-99-57-71-113-147-97-71-115 99-163-128-114-147-115-71-97-147-113-71-57 97-147-

## 1.2.1 Applying LeaderboardCyclopeptideSequencing on the extended amino acid alphabet 

(i.e., every integer between 57 and 200 inclusively) to Spectrum10 with N = 1000 returns 34 different linear peptides of maximum score. What are they? (Return your answer in integer format separated by a space, e.g., 113-147-71-129 199-200-61.)

In [14]:
def LeaderBoardTop(n, spectrum):

  def LinearSpectrum(peptide):
    #print('peptide', peptide)
    
    lPeptide = len(peptide)
    prefixMass = [0] * (lPeptide) 

    for i in range(0, lPeptide):
      for symbol in tbAAtoMass.keys():
        if symbol == peptide[i]:
          prefixMass[i] = prefixMass[i-1] + tbAAtoMass[symbol]
    prefixMass.insert(0,0)
    
    linearSpectrum = [0]
    c = 0
    for i in range(0, len(prefixMass)-1):
      for j in range(i+1, len(prefixMass)):
        c +=1
        linearSpectrum.append(prefixMass[j]-prefixMass[i])

    linearSpectrum.sort()
    #print(linearSpectrum)
    return linearSpectrum

  def Cyclospectrum(peptide):
 
    part1 = LinearSpectrum(peptide=peptide[:-1])
    pref1 = LinearSpectrum(peptide=peptide)
    tMass=pref1[-1] # mass of the full element
    part2 = [tMass - x for x in part1]

    final = part1 + part2
    final.sort()
    return final


  def GetAminoAcidMassTable():
    ''' Returns dictionary in format aminoacid: integer mass  
    '''
    with open('/content/drive/MyDrive/Colab Notebooks/Bionformatics-coursera-sandiego/Course 2/integer_mass_table.txt') as f:
      tb = f.readlines()
    massTable = dict()
    for line in tb:
      massTable[line.split('\n')[0].split(' ')[0]] = int(line.split('\n')[0].split(' ')[1])
    return massTable


  def GetMassAminoAcidTable():
    ''' Returns dictionary in format integer mass : aminoacid
    '''
    with open('/content/drive/MyDrive/Colab Notebooks/Bionformatics-coursera-sandiego/Course 2/integer_mass_table.txt') as f:
      tb = f.readlines()
    massTable = dict()
    for line in tb:
      massTable[int(line.split('\n')[0].split(' ')[1])] = line.split('\n')[0].split(' ')[0]
    return massTable


  def Expand(candidatePeptides):
      ''' Returns newCollection containing all possible extensions of peptides in 
          candidatePeptides by a single amino acid mass
      ''' 
      newCollection = set()
      if candidatePeptides != {''}:
        
        [newCollection.add(candidate + '-' + str(tbAAtoMass[peptide])) 
                          for candidate in candidatePeptides \
                            for peptide in tbAAtoMass ]
      else:
        [newCollection.add(candidate + str(tbAAtoMass[peptide])) \
                          for candidate in candidatePeptides \
                          for peptide in tbAAtoMass ]
                        
      return newCollection


  def GetParentMass(spectrum):
    
    return spectrum[-1]
  

  def GetMass(peptide):
    ''' Input: subpeptide string
        Output: mass of subpeptidestring
    '''
    mass = 0 
    
    for element in peptide.split('-'):
      if element != '':
        mass = mass + int(element)

    return mass 


  def PeptToAA(peptide):
    ''' input: aminoacid sequence in format 137-57-128
        output: aminoacid
    '''
    aminoAcid = ''
    for aa in peptide.split('-'):
      aminoAcid += tbMasstoAA[int(aa)]
    return aminoAcid 


  def PeptideConsistentWithSpectrum(peptide):
    ''' case 1: peptide is string '128'
        case 2: peptide is string '128-113-99'
    '''
    peptideAA = PeptToAA(peptide) 
    LS = LinearSpectrum(peptide=peptideAA, alphabet=massTable.keys(), aminoAcidMass=massTable)

    spectrumCopy = spectrum.copy()
    cnt = 0

    for element in LS:
      if int(element) in spectrumCopy:
        cnt += 1 
        spectrumCopy.remove(int(element))
      else:
        return False
    if cnt == len(LS):
      return True
      

  def LinearScore(peptide, spectrum):
    LS = LinearSpectrum(peptide)
    CP = spectrum.copy()
    score = 0
    #print('LS', peptide, LS)

    if peptide != '':
      for s in LS:
        if (s in CP):
          score += 1 
          CP.remove(s)

    return score

  def CycloScore(peptide, spectrum):
    LS = Cyclospectrum(peptide)
    CP = spectrum.copy()
    score = 0
    #print('LS', peptide, LS)

    if peptide != '':
      for s in LS:
        if (s in CP):
          score += 1 
          CP.remove(s)

    return score


  def Trim(leaderBoard, spectrum, n):
    ''' leaderBoard: set of peptides {'99-99-99', ''}
        spectrum: list of int [0,99,128]
        n = 10 
    '''
    scores = []
    if len(leaderBoard) > n:
      
      trimedLeaderBoard = set()

      [scores.append(LinearScore(PeptToAA(peptide), spectrum)) for peptide in leaderBoard]
      #print('scores', sorted(scores,reverse=True))
      
      sortedLB = [l for _,l in sorted(zip(scores,leaderBoard))]
      #print('sortedLB', sortedLB)
      
      scores = sorted(scores)
      #print('sorted scores', scores)

      part1 = sortedLB[-n:]
      #print('part1', part1)
      
      part2 = []
      i = 1
      while len(scores)>(n+i) and (scores[-n]==scores[-n-i]):
        part2.append(sortedLB[-n-i])
        i += 1
      trimedLeaderBoard = set(part1 + part2)
    else: 
      trimedLeaderBoard = leaderBoard
    #print('trimedLeaderBoard ', len(leaderBoard), '->',len(trimedLeaderBoard),  trimedLeaderBoard )
    #print('scores',scores)
    return trimedLeaderBoard 


  # *** Main *** 

  tbAAtoMass = GetAminoAcidMassTable()
  tbMasstoAA = GetMassAminoAcidTable()
  parentMass = GetParentMass(spectrum)

  leaderBoard = {''} # set of strings
  leaderPeptide = ''

  topScore = 0
  topPeptides = []
  topScores = [] 


  while len(leaderBoard) > 0:
    leaderBoardExp = Expand(leaderBoard)
    leaderBoardCopy = leaderBoardExp.copy()

    for peptide in leaderBoardExp:  # peptide 137-57-128  
      peptideMass = GetMass(peptide)
      if peptideMass == parentMass:
        CScore1 = CycloScore(PeptToAA(peptide), spectrum)
        CScore2 = CycloScore(leaderPeptide, spectrum)
      
        if CScore1 == topScore:
          topPeptides.append(peptide) ###
          topScores.append(topScore)
        if topScore < CScore1:
          topPeptides = [peptide]
          topScores = [CScore1]
          topScore = CScore1
      
        if  CScore1 > CScore2 :
          leaderPeptide = peptide 
      elif peptideMass > parentMass:
        leaderBoardCopy.remove(peptide)
    leaderBoard = Trim(leaderBoardCopy, spectrum, n)
    

  return topPeptides , leaderPeptide, PeptToAA(leaderPeptide), CycloScore(PeptToAA(leaderPeptide), spectrum)

n = 10 
spectrum = [int(s) for s in '0 71 113 129 147 200 218 260 313 331 347 389 460'.split(' ')]
LB = LeaderBoardTop(n, spectrum)
print(LB)

(['71-129-113-147', '129-71-147-113', '113-129-71-147', '147-71-129-113', '113-147-71-129', '71-147-113-129', '129-113-147-71', '147-113-129-71'], '147-113-129-71', 'FLEA', 13)


## code

In [4]:
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving dataset_103_2.txt to dataset_103_2.txt
User uploaded file "dataset_103_2.txt" with length 378 bytes


In [35]:
with open(fn) as f:
  data = f.readlines() 

n2 = int(data[0].split('\n')[0])
spectrum2 = data[1].split('\n')[0]
spectrum2 = [int(x) for x in spectrum2.split(' ') if x != '']

In [36]:
R = LeaderBoardTop(n2, spectrum2)

In [37]:
R

(['147-113-128-99-163-128-114-147-186-97',
  '114-128-163-99-128-113-147-97-186-147',
  '97-147-113-128-99-163-128-114-147-186',
  '186-147-114-128-163-99-128-113-147-97',
  '128-163-99-128-113-147-97-186-147-114',
  '128-114-147-186-97-147-113-128-99-163',
  '113-128-99-163-128-114-147-186-97-147',
  '163-99-128-113-147-97-186-147-114-128',
  '128-99-163-128-114-147-186-97-147-113',
  '99-128-113-147-97-186-147-114-128-163',
  '97-186-147-114-128-163-99-128-113-147',
  '128-113-147-97-186-147-114-128-163-99',
  '99-163-128-114-147-186-97-147-113-128',
  '114-147-186-97-147-113-128-99-163-128',
  '147-186-97-147-113-128-99-163-128-114',
  '163-128-114-147-186-97-147-113-128-99',
  '147-97-186-147-114-128-163-99-128-113',
  '186-97-147-113-128-99-163-128-114-147',
  '113-147-97-186-147-114-128-163-99-128',
  '147-114-128-163-99-128-113-147-97-186',
  '114-147-186-97-147-113-128-99-163-57-71',
  '163-128-114-147-129-57-97-147-113-128-99',
  '114-128-163-99-128-113-147-97-71-115-147',
  '

In [40]:
len(set(R[0]))

240

In [41]:
s = ''
for el in R[0]:
  s += str(el) + ' '
s 

'147-113-128-99-163-128-114-147-186-97 114-128-163-99-128-113-147-97-186-147 97-147-113-128-99-163-128-114-147-186 186-147-114-128-163-99-128-113-147-97 128-163-99-128-113-147-97-186-147-114 128-114-147-186-97-147-113-128-99-163 113-128-99-163-128-114-147-186-97-147 163-99-128-113-147-97-186-147-114-128 128-99-163-128-114-147-186-97-147-113 99-128-113-147-97-186-147-114-128-163 97-186-147-114-128-163-99-128-113-147 128-113-147-97-186-147-114-128-163-99 99-163-128-114-147-186-97-147-113-128 114-147-186-97-147-113-128-99-163-128 147-186-97-147-113-128-99-163-128-114 163-128-114-147-186-97-147-113-128-99 147-97-186-147-114-128-163-99-128-113 186-97-147-113-128-99-163-128-114-147 113-147-97-186-147-114-128-163-99-128 147-114-128-163-99-128-113-147-97-186 114-147-186-97-147-113-128-99-163-57-71 163-128-114-147-129-57-97-147-113-128-99 114-128-163-99-128-113-147-97-71-115-147 99-163-128-114-147-186-97-147-113-57-71 128-113-147-97-186-147-57-57-128-163-99 128-113-147-97-186-147-114-57-71-163-

## 1.3.1. Spectral Convolution Problem: Compute the convolution of a spectrum.


Input: A collection of integers Spectrum in increasing order.

Output: The list of elements in the convolution of Spectrum. If an element has multiplicity k, it should appear exactly k times; you may return the elements in any order.

In [29]:
def DataToString(data):
  s = ''
  for el in data:
    s += str(int(el)) + ' '
  return s[:-1]
def ConvolutionSpectrum(spectrum):
  spectrum = np.array(sorted(spectrum))
  data = np.array([])
  for i in range(0, len(spectrum)-1):
    lst = spectrum[i+1:] - spectrum[i]
    data = np.append(data, lst)
  data = data[data != 0]
  return DataToString(data)

spectrum = [0, 137, 186, 323]
ConvolutionSpectrum(spectrum)

'137 186 323 49 186 137'

### Stepik submission

In [30]:
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving dataset_104_4-3.txt to dataset_104_4-3.txt
User uploaded file "dataset_104_4-3.txt" with length 362 bytes


In [31]:
with open(fn) as f:
  spectr = f.readline().split('\n')[0]
spectr = [int(s) for s in spectr.split(' ')]
print(spectr)

[0, 71, 71, 87, 99, 99, 113, 115, 128, 128, 131, 142, 170, 186, 186, 212, 215, 228, 230, 241, 256, 257, 257, 259, 299, 314, 327, 328, 343, 343, 356, 358, 370, 385, 387, 398, 442, 443, 456, 458, 469, 469, 471, 474, 486, 513, 529, 556, 568, 571, 573, 573, 584, 586, 599, 600, 644, 655, 657, 672, 684, 686, 699, 699, 714, 715, 728, 743, 783, 785, 785, 786, 801, 812, 814, 827, 830, 856, 856, 872, 900, 911, 914, 914, 927, 929, 943, 943, 955, 971, 971, 1042]


In [33]:
ConvolutionSpectrum(spectr)

'71 71 87 99 99 113 115 128 128 131 142 170 186 186 212 215 228 230 241 256 257 257 259 299 314 327 328 343 343 356 358 370 385 387 398 442 443 456 458 469 469 471 474 486 513 529 556 568 571 573 573 584 586 599 600 644 655 657 672 684 686 699 699 714 715 728 743 783 785 785 786 801 812 814 827 830 856 856 872 900 911 914 914 927 929 943 943 955 971 971 1042 16 28 28 42 44 57 57 60 71 99 115 115 141 144 157 159 170 185 186 186 188 228 243 256 257 272 272 285 287 299 314 316 327 371 372 385 387 398 398 400 403 415 442 458 485 497 500 502 502 513 515 528 529 573 584 586 601 613 615 628 628 643 644 657 672 712 714 714 715 730 741 743 756 759 785 785 801 829 840 843 843 856 858 872 872 884 900 900 971 16 28 28 42 44 57 57 60 71 99 115 115 141 144 157 159 170 185 186 186 188 228 243 256 257 272 272 285 287 299 314 316 327 371 372 385 387 398 398 400 403 415 442 458 485 497 500 502 502 513 515 528 529 573 584 586 601 613 615 628 628 643 644 657 672 712 714 714 715 730 741 743 756 759 785 785

In [ ]:
#spectrum = np.array(spectrum)
spectrum = spectr
import pandas as pd
df = pd.DataFrame(index = spectrum, columns = spectrum[:-1] )

for i in range(0, len(spectrum)-1):
    df.iloc[i+1:, i] = df.index[i+1:] - df.index[i]  #spectrum[i+1:] - spectrum[i]
df

## 1.3.2 Code Challenge: Implement ConvolutionCyclopeptideSequencing.

Input: An integer M, an integer N, and a collection of (possibly repeated) integers Spectrum.

Output: A cyclic peptide LeaderPeptide with amino acids taken only from the top M elements (and ties) of the convolution of Spectrum that fall between 57 and 200, and where the size of Leaderboard is restricted to the top N (and ties).


In [121]:
def LinearSpectrum(peptide):
  ''' input: peptide presented in masses 
      output: spectrum in masses
  ''' 
  linearSpectrum = [0]
  if peptide != '':
    peptide = [int(x) for x in peptide.split('-')]

    
    lPeptide = len(peptide)
    prefixMass = [0] * (lPeptide) 

    for i in range(0, lPeptide):
      for symbol in peptide: 
        if symbol == peptide[i]:
          prefixMass[i] = prefixMass[i-1] + symbol
    prefixMass.insert(0,0)
    
    #linearSpectrum = [0]
    c = 0
    for i in range(0, len(prefixMass)-1):
      for j in range(i+1, len(prefixMass)):
        c +=1
        linearSpectrum.append(prefixMass[j]-prefixMass[i])

    linearSpectrum.sort()
  return linearSpectrum

def Cyclospectrum(peptide):

  part1 = LinearSpectrum(peptide=peptide[:-1])
  pref1 = LinearSpectrum(peptide=peptide)
  tMass=pref1[-1] # mass of the full element
  part2 = [tMass - x for x in part1]

  final = part1 + part2
  final.sort()
  return final


def Expand(candidatePeptides, alphabetMass):
    ''' Returns newCollection containing all possible extensions of peptides in 
        candidatePeptides by a single amino acid mass
    ''' 
    newCollection = set()
    if candidatePeptides != {''}:
      
      [newCollection.add(candidate + '-' + str(peptide)) 
                        for candidate in candidatePeptides \
                          for peptide in alphabetMass ]
    else:
      [newCollection.add(candidate + str(peptide)) \
                        for candidate in candidatePeptides \
                        for peptide in alphabetMass ]
                      
    return newCollection


def GetParentMass(spectrum):
  
  return spectrum[-1]


def GetMass(peptide):
  ''' Input: subpeptide string
      Output: mass of subpeptidestring
  '''
  mass = 0 
  
  for element in peptide.split('-'):
    if element != '':
      mass = mass + int(element)

  return mass 



def PeptideConsistentWithSpectrum(peptide):
  ''' case 1: peptide is string '128'
      case 2: peptide is string '128-113-99'
  '''
  LS = LinearSpectrum(peptide=peptide, alphabet=massTable.keys(), aminoAcidMass=massTable)

  spectrumCopy = spectrum.copy()
  cnt = 0

  for element in LS:
    if int(element) in spectrumCopy:
      cnt += 1 
      spectrumCopy.remove(int(element))
    else:
      return False
  if cnt == len(LS):
    return True
    

def LinearScore(peptide, spectrum):
  LS = LinearSpectrum(peptide)
  CP = spectrum.copy()
  score = 0
  #print('LS', peptide, LS)

  if peptide != '':
    for s in LS:
      if (s in CP):
        score += 1 
        CP.remove(s)

  return score

def CycloScore(peptide, spectrum):
  LS = Cyclospectrum(peptide)
  CP = spectrum.copy()
  score = 0
  #print('LS', peptide, LS)

  if peptide != '':
    for s in LS:
      if (s in CP):
        score += 1 
        CP.remove(s)

  return score


def Trim(leaderBoard, spectrum, n):
  ''' leaderBoard: set of peptides {'99-99-99', ''}
      spectrum: list of int [0,99,128]
      n = 10 
  '''
  scores = []
  if len(leaderBoard) > n:
    
    trimedLeaderBoard = set()

    [scores.append(LinearScore(peptide, spectrum) ) for peptide in leaderBoard]
    #print('scores', sorted(scores,reverse=True))
    
    sortedLB = [l for _,l in sorted(zip(scores,leaderBoard))]
    #print('sortedLB', sortedLB)
    
    scores = sorted(scores)
    #print('sorted scores', scores)

    part1 = sortedLB[-n:]
    #print('part1', part1)
    
    part2 = []
    i = 1
    while len(scores)>(n+i) and (scores[-n]==scores[-n-i]):
      part2.append(sortedLB[-n-i])
      i += 1
    trimedLeaderBoard = set(part1 + part2)
  else: 
    trimedLeaderBoard = leaderBoard
  #print('trimedLeaderBoard ', len(leaderBoard), '->',len(trimedLeaderBoard),  trimedLeaderBoard )
  #print('scores',scores)
  return trimedLeaderBoard 

def ConvolutionSpectrum(spectrum):
  ''' input: int list
      output: np array 
  '''
  spectrum = np.array(sorted(spectrum))
  data = np.array([])
  for i in range(0, len(spectrum)-1):
    lst = spectrum[i+1:] - spectrum[i]
    data = np.append(data, lst)
  data = data[data != 0]
  #data = [int(x) for x in data]
  return data

def GetMassAlphabet(spectrum, M, N):

  conv = ConvolutionSpectrum(spectrum)
  conv = conv[conv < 200]
  conv = conv[conv >= 57]

  lst = Counter(conv).most_common(M)
  frequency = List[-1][1]

  massAlphabet = sorted( [int(k) for k, v in dict(Counter(conv)).items() if v >= frequency] )

  return massAlphabet


def LeaderBoardCustomAlphabet(N,M, spectrum):
  alphabetMass = GetMassAlphabet(spectrum, M, N)

  parentMass = GetParentMass(spectrum)

  leaderBoard = {''} # set of strings
  leaderPeptide = ''

  topScore = 0
  topPeptides = []
  topScores = [] 

  while len(leaderBoard) > 0:
    leaderBoardExp = Expand(leaderBoard,alphabetMass)
    leaderBoardCopy = leaderBoardExp.copy()

    for peptide in leaderBoardExp:  # peptide 137-57-128  
      peptideMass = GetMass(peptide)
      if peptideMass == parentMass:
        CScore1 = CycloScore(peptide, spectrum)
        CScore2 = CycloScore(leaderPeptide, spectrum)
      
        if CScore1 == topScore:
          topPeptides.append(peptide) ###
          topScores.append(topScore)
        if topScore < CScore1:
          topPeptides = [peptide]
          topScores = [CScore1]
          topScore = CScore1
      
        if  CScore1 > CScore2 :
          leaderPeptide = peptide 
      elif peptideMass > parentMass:
        leaderBoardCopy.remove(peptide)
    leaderBoard = Trim(leaderBoardCopy, spectrum, n)
    
  print('There are', len(topPeptides), 'top peptides with score', CycloScore(leaderPeptide,spectrum))

  return leaderPeptide , topPeptides #leaderPeptide, , CycloScore(PeptToAA(leaderPeptide), spectrum)

# n = 10 
# spectrum = [int(s) for s in '0 71 113 129 147 200 218 260 313 331 347 389 460'.split(' ')]
# LB = LeaderBoardCustomAlphabet(N, M, spectrum)
# print(LB)

M = 20
N = 60
s = '57 57 71 99 129 137 170 186 194 208 228 265 285 299 307 323 356 364 394 422 493'.split(' ')
spectrum = [int(x) for x in s]

LeaderBoardCustomAlphabet(N, M, spectrum)

There are 11 top peptides with score 20


('99-129-57-137-71',
 ['99-129-57-137-71',
  '99-71-137-57-129',
  '71-137-57-129-99',
  '129-57-137-71-99',
  '57-137-71-99-129',
  '71-99-129-57-137',
  '57-129-99-71-137',
  '137-57-129-99-71',
  '71-99-129-57-79-58',
  '57-137-71-99-58-71',
  '99-71-137-57-58-71'])

### ExtraDataset

In [ ]:
url = 'http://bioinformaticsalgorithms.com/data/extradatasets/antibiotics/convolution_cyclopeptide_sequencing.txt'


### Stepik submission

In [100]:
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  

Saving dataset_104_7-2.txt to dataset_104_7-2.txt
User uploaded file "dataset_104_7-2.txt" with length 814 bytes


In [101]:
with open(fn) as f:
  M = int(f.readline().split('\n')[0])
  N = int(f.readline().split('\n')[0])
  spectrum = f.readline().split('\n')[0]
spectrum = [int(s) for s in spectrum.split(' ')]

print(spectrum)
print(N)
print(M)

[0, 103, 103, 103, 113, 113, 113, 114, 114, 115, 128, 128, 137, 137, 186, 217, 217, 226, 226, 227, 231, 240, 240, 242, 243, 252, 265, 289, 299, 320, 339, 340, 346, 354, 355, 355, 368, 368, 379, 380, 402, 403, 412, 453, 457, 468, 469, 482, 483, 483, 483, 492, 496, 506, 515, 516, 525, 572, 581, 586, 595, 597, 605, 610, 611, 619, 620, 628, 629, 639, 643, 700, 700, 708, 718, 723, 723, 725, 732, 742, 742, 748, 756, 758, 767, 803, 821, 836, 837, 838, 845, 851, 856, 862, 869, 870, 871, 886, 904, 940, 949, 951, 959, 965, 965, 975, 982, 984, 984, 989, 999, 1007, 1007, 1064, 1068, 1078, 1079, 1087, 1088, 1096, 1097, 1102, 1110, 1112, 1121, 1126, 1135, 1182, 1191, 1192, 1201, 1211, 1215, 1224, 1224, 1224, 1225, 1238, 1239, 1250, 1254, 1295, 1304, 1305, 1327, 1328, 1339, 1339, 1352, 1352, 1353, 1361, 1367, 1368, 1387, 1408, 1418, 1442, 1455, 1464, 1465, 1467, 1467, 1476, 1480, 1481, 1481, 1490, 1490, 1521, 1570, 1570, 1579, 1579, 1592, 1593, 1593, 1594, 1594, 1594, 1604, 1604, 1604, 1707]
390
17


In [102]:
LeaderBoardCustomAlphabet(N, M, spectrum)

'113-113-114-128-137-103-128-115-137-103-114-103-186-113'

In [103]:
url = 'http://bioinformaticsalgorithms.com/data/realdatasets/Antibiotics/Tyrocidine_B1_Spectrum_25.txt'
with urllib.request.urlopen(url) as f:
    data_25 = f.read().decode('utf-8')

data_25 = [int(x) for x in data_25.split(' ')]

data_25[:10]

[0, 97, 99, 113, 114, 115, 128, 128, 147, 147]

In [123]:
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving dataset_104_8.txt to dataset_104_8 (4).txt
User uploaded file "dataset_104_8.txt" with length 401 bytes
[0, 97, 99, 113, 114, 115, 128, 128, 147, 147, 163, 186, 227, 241, 242, 244, 244, 256, 260, 261, 262, 283, 291, 309, 330, 333, 340, 347, 385, 388, 389, 390, 390, 405, 435, 447, 485, 487, 503, 504, 518, 544, 552, 575, 577, 584, 599, 608, 631, 632, 650, 651, 653, 672, 690, 691, 717, 738, 745, 770, 779, 804, 818, 819, 827, 835, 837, 875, 892, 892, 917, 932, 932, 933, 934, 965, 982, 989, 1039, 1060, 1062, 1078, 1080, 1081, 1095, 1136, 1159, 1175, 1175, 1194, 1194, 1208, 1209, 1223, 1322]
1000
20


In [126]:
with open(fn) as f:
  M = int(f.readline().split('\n')[0])
  N = int(f.readline().split('\n')[0])
  spectrum = f.readline().split('\n')[0]
spectrum = [int(s) for s in spectrum.split(' ')]


leaderPeptide , topPeptides = LeaderBoardCustomAlphabet(N, M, spectrum)
print(topPeptides)

There are 3 top peptides with score 83
['147-114-128-163-99-128-113-147-97-186', '163-128-114-147-186-97-147-113-128-99', '147-113-128-99-163-128-114-147-97-89-97']
